In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yaml
%matplotlib inline

try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

2. Loading Data

In [ ]:
# config
#['data']['clean']['file_demo_experiment_cleaned']

In [20]:
df_client = pd.read_csv(config['data']['raw']['file_client_profile'])
df_experiment = pd.read_csv(config['data']['raw']['file_experiment_clients'])
df_web1 = pd.read_csv(config['data']['raw']['file_web1'])
df_web2 = pd.read_csv(config['data']['raw']['file_web2'])
df_client_experiment = pd.read_csv(config['data']['raw']['file_demo_experiment'])

C:\Users\Gast\AppData\Local\Temp\ipykernel_22188\325902528.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_client_experiment = pd.read_csv(config['data']['raw']['file_demo_experiment'])


In [ ]:
df_footprints = pd.concat([df_web1, df_web2])
df_footprints.to_csv(config['data']['raw']['file_footprints'])
df_footprints

3. Generic EDA

In [ ]:
df_client.info()

In [ ]:
df_client.head()
# df_client.isna().sum()
# df_client["gendr"].unique()
# df_client["bal"].min()
# df_client["bal"].max()


In [ ]:
df_client["calls_6_mnth"].unique()


In [ ]:
# df_client["num_accts"] =df_client["num_accts"].astype(int)

In [ ]:
df_experiment.head()

In [ ]:
df_experiment["Variation"].value_counts()
# df_experiment.dtypes
# df_experiment.isna().sum()

In [ ]:
df_footprints.info()
df_footprints.head()

In [ ]:
df_footprints["visitor_id"].nunique()

In [ ]:
# df_web1.head()
df_web1.info()
df_web1.duplicated().sum()

In [ ]:
# df_web2.head()
df_web2.info()
df_web2.duplicated().sum()

In [ ]:
# df_footprints["process_step"].unique()
# df_footprints["client_id"].value_counts()
# df_footprints["process_step"].unique()
df_footprints.isna().sum()

3a. df_client

In [ ]:
# df_client.info()
df_client.head()

In [ ]:
df_client_experiment.info()
df_client_experiment.head()


In [ ]:
Active = ['Test', 'Control']
test = df_client_experiment[df_client_experiment['variation'].isin(Active)]
test['clnt_age'].describe()

In [ ]:
test.head()

In [ ]:
test['clnt_tenure_yr'].describe()

In [ ]:
# sns.histplot(data=test,x=test["clnt_age"],y=test[""], )
sns.histplot(data=test["clnt_age"], kde=False, bins=10, color="salmon");

In [ ]:
sns.histplot(data=test["clnt_tenure_yr"], kde=False, bins=20, color="salmon");

In [ ]:
test["clnt_age"].value_counts().sort_values(ascending = False)

In [ ]:
test[test["clnt_age"]>35]

In [ ]:
df_client.select_dtypes("number").nunique().sort_values(ascending=False)

In [ ]:
df_client.select_dtypes("object").nunique().sort_values(ascending=False)

In [ ]:
potential_categorical_from_numerical = df_client.select_dtypes("number").loc[:, df_client.select_dtypes("number").nunique() < 20]
potential_categorical_from_numerical

In [ ]:
df_client["gendr"].value_counts()
# df_client.isna().sum()

In [ ]:
df_client["clnt_tenure_yr"].value_counts().sort_index(ascending=True)#.tail()
min_clnt_tenure_yr = df_client["clnt_tenure_yr"].min()
max_clnt_tenure_yr = df_client["clnt_tenure_yr"].max()
avg_clnt_tenure_yr = df_client["clnt_tenure_yr"].mean()
min_clnt_tenure_yr, max_clnt_tenure_yr

In [ ]:
df_client["clnt_tenure_mnth"].value_counts().sort_index(ascending=True)#.tail()

In [ ]:
df_client["clnt_age"].value_counts().sort_index(ascending=True)#.tail()

4. Clean

In [ ]:
# 
# df_client_experiment.isna().sum()
# df1_demo.shape

In [22]:
# Clean/rename columns
df_client_experiment = df_client_experiment.rename(columns = {"clnt_tenure_yr": "client_tenure_year", "clnt_tenure_mnth": "client_tenure_month", "clnt_age" : "client_age", "gendr": "gender", "bal": "balance","Variation": "treatment_group"}).drop(columns = "Unnamed: 0")
df_client_experiment.columns

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
# df_client_experiment.drop("Unnamed:\ 0")

In [ ]:
# df_client_experiment["client_tenure_year"].unique()

In [ ]:
# df_client_experiment[df_client_experiment["client_tenure_month"].isna()]
# df_client_experiment.shape

In [23]:
#Drop  row, when values in subset are null
df_client_experiment = df_client_experiment.dropna(subset = ["client_tenure_year", "client_tenure_month", "client_age"], how = "all" )
df_client_experiment.shape

(70595, 10)

In [ ]:
# df_client_experiment.tail()

In [ ]:
# df_client_experiment.isna().sum()

In [ ]:
# df_client_experiment.info()

In [24]:
df_client_experiment = df_client_experiment.dropna(subset = "variation")
df_client_experiment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50488 entries, 0 to 50499
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   client_id            50488 non-null  int64  
 1   client_tenure_year   50488 non-null  float64
 2   client_tenure_month  50488 non-null  float64
 3   client_age           50487 non-null  float64
 4   gender               50488 non-null  object 
 5   num_accts            50488 non-null  float64
 6   balance              50488 non-null  float64
 7   calls_6_mnth         50488 non-null  float64
 8   logons_6_mnth        50488 non-null  float64
 9   variation            50488 non-null  object 
dtypes: float64(7), int64(1), object(2)
memory usage: 4.2+ MB


In [25]:
df_client_experiment["client_age"] = df_client_experiment["client_age"].fillna(df_client_experiment["client_age"].mean())
df_client_experiment.isna().sum()

client_id              0
client_tenure_year     0
client_tenure_month    0
client_age             0
gender                 0
num_accts              0
balance                0
calls_6_mnth           0
logons_6_mnth          0
variation              0
dtype: int64

In [26]:
def clean_gdr(x):
    gender = str(x)
    if gender == "X": 
        return "U"
    else:
        return x
    
df_client_experiment["gender"] = df_client_experiment["gender"].fillna("U")
# df_client["gendr"] = df_client["gendr"].apply(lambda x:"U" if x == "X" else x for x in df_client["gendr"])
df_client_experiment["gender"] = df_client_experiment["gender"].str.strip().apply(clean_gdr)
df_client_experiment["gender"].unique() 

array(['U', 'M', 'F'], dtype=object)

In [ ]:
# df_client_experiment.info()

In [27]:
for col in ["client_tenure_year","client_tenure_month", "client_age", "num_accts","calls_6_mnth", "logons_6_mnth"]:
    df_client_experiment[col] = df_client_experiment[col].astype("int64")
df_client_experiment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50488 entries, 0 to 50499
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   client_id            50488 non-null  int64  
 1   client_tenure_year   50488 non-null  int64  
 2   client_tenure_month  50488 non-null  int64  
 3   client_age           50488 non-null  int64  
 4   gender               50488 non-null  object 
 5   num_accts            50488 non-null  int64  
 6   balance              50488 non-null  float64
 7   calls_6_mnth         50488 non-null  int64  
 8   logons_6_mnth        50488 non-null  int64  
 9   variation            50488 non-null  object 
dtypes: float64(1), int64(7), object(2)
memory usage: 4.2+ MB


In [ ]:
# df_client_experiment

In [28]:
df_client_experiment.to_csv(config['data']['clean']['file_demo_experiment_cleaned'],sep=";")